In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform),
    batch_size=64, shuffle=False
)


100%|██████████| 9.91M/9.91M [00:01<00:00, 6.34MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 350kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.75MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.72MB/s]


In [3]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(28*28, 128),
#             nn.ReLU(),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.Linear(64, 10)
#         )

#     def forward(self, x):
#         return self.fc(x)

# model = Net().to(device)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # (batch, 1, 28, 28) -> (batch, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> (batch, 32, 14, 14)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # -> (batch, 64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> (batch, 64, 7, 7)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc(x)
        return x

model = CNN().to(device)



In [5]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
losses = []

for epoch in range(5):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Accuracy batch
        preds = output.argmax(dim=1)
        correct += (preds == target).sum().item()
        total += target.size(0)

    avg_loss = epoch_loss / len(train_loader)
    accuracy = 100. * correct / total
    losses.append({'epoch': epoch+1, 'loss': avg_loss, 'accuracy': accuracy})
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f} | Accuracy = {accuracy:.2f}%")


Epoch 1: Loss = 0.0309 | Accuracy = 98.97%
Epoch 2: Loss = 0.0192 | Accuracy = 99.38%
Epoch 3: Loss = 0.0155 | Accuracy = 99.45%
Epoch 4: Loss = 0.0105 | Accuracy = 99.64%
Epoch 5: Loss = 0.0093 | Accuracy = 99.69%


In [6]:
df = pd.DataFrame(losses)
df.to_csv("training_losses.csv", index=False)
df


,epoch,loss,accuracy
0,1,0.030930,98.971667
1,2,0.019244,99.378333
2,3,0.015504,99.448333
3,4,0.010528,99.645000
4,5,0.009289,99.691667


In [9]:
import torch.onnx

dummy_input = torch.randn(1, 1, 28, 28).to(device)
torch.onnx.export(model, dummy_input, "mnist_model.onnx",
                  input_names=["input"], output_names=["output"],
                  dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print("Modèle exporté en mnist_model.onnx ✅")


Modèle exporté en mnist_model.onnx ✅


In [10]:
import onnx

model = onnx.load("mnist_model.onnx")
print([input.name for input in model.graph.input])


['input']


In [12]:
import onnxruntime as ort
import numpy as np
from torchvision import datasets, transforms

# Charger le modèle ONNX
session = ort.InferenceSession("mnist_model.onnx")

# Préparation de la transformation identique à celle du training
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Ajuste selon ton training !
])

# Charger jeu de test MNIST (download si besoin)
test_dataset = datasets.MNIST(root='.', train=False, download=True, transform=transform)

# Prendre une image test avec son label (exemple: index 1)
img, label = test_dataset[1]

# Préparer l'entrée pour le modèle ONNX
input_name = session.get_inputs()[0].name

# MNIST : batch_size=1, canaux=1, hauteur=28, largeur=28
input_data = img.numpy().reshape(1, 1, 28, 28).astype(np.float32)

# Inférence
outputs = session.run(None, {input_name: input_data})

# Résultat brut (logits)
logits = outputs[0]

# Affichage des logits et prédiction finale
print("Logits:", logits)
predicted_label = np.argmax(logits)
print(f"Prédiction du modèle : {predicted_label}")
print(f"Label réel : {label}")


Logits: [[  1.4271472   1.4908273  19.079624  -11.489559   -6.413771  -22.664413
   -1.2811205 -10.712002   -4.3989377 -12.344233 ]]
Prédiction du modèle : 2
Label réel : 2


In [14]:
#TEST COMPLET
import onnxruntime as ort
import numpy as np

# Charger le modèle ONNX
session = ort.InferenceSession("mnist_model.onnx")

# Fonction d'inférence
def evaluate_onnx(session, test_loader):
    input_name = session.get_inputs()[0].name
    correct = 0
    total = 0

    for data, target in test_loader:
        input_data = data.numpy().astype(np.float32)
        outputs = session.run(None, {input_name: input_data})
        preds = np.argmax(outputs[0], axis=1)
        correct += (preds == target.numpy()).sum()
        total += target.size(0)

    accuracy = 100. * correct / total
    print(f"Accuracy ONNX sur test_loader : {accuracy:.2f}%")

evaluate_onnx(session, test_loader)


Accuracy ONNX sur test_loader : 99.15%
